<font face="微软雅黑">
1.导入python处理数据的常用包

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame,Series

2.读取csv格式的原始数据文件，大致浏览一下数据情况

In [33]:
data_train= pd.read_csv('C:/Users/Seele/Documents/Jupyter/KAGGLE/Titanic/train.csv')
data_train.shape

(891, 12)

3.查看表中数据的一些信息
可以发现，数据为891*12的DataFrame格式。但是文件中数据有部分缺失值，其中Age只有714行，且cabin一栏仅204行，缺失严重。

4.对表中数据进行初步处理。
表中passengerid为无意义值，可直接删除。Cabin过于复杂且缺失值过多，可考虑先删除。

In [34]:
data_train = data_train.drop(['Cabin','PassengerId','Ticket'],1)

现在，我们拥有8个特征值，仔细观察这些特征值，名字与船舱等级和存活与否没有多大的联系，可以将姓名一栏删除。

In [35]:
data_train = data_train.drop(['Name'],1)

In [36]:
data_train['Age'].max()-data_train['Age'].min()

79.579999999999998

In [37]:
data_train['Fare'].max()- data_train['Fare'].min()

512.32920000000001

In [38]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


5.Embarked 和Age 列缺失值数量在可接受范围内。用Embarked的出现频率最高的和Age的中位数填充缺失值。

In [39]:

data_train['Embarked']= data_train['Embarked'].fillna('S')

6.将性别二值化

In [40]:
data_train.loc[data_train['Sex'] == 'male','Sex'] = 0
data_train.loc[data_train['Sex'] == 'female','Sex'] =1

7.将Embarked类别用不同数值代替

In [41]:

data_train.loc[data_train['Embarked'] == 'S', 'Embarked'] = 0
data_train.loc[data_train['Embarked'] == 'C', 'Embarked'] = 1
data_train.loc[data_train['Embarked'] == 'Q', 'Embarked'] = 2

In [42]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


8.利用随机森林给出年龄缺失值的预测值

In [43]:
from sklearn.ensemble import RandomForestRegressor

In [44]:
def set_missing_ages(data_train):
    age_data = data_train[['Age','Pclass', 'Sex', 'SibSp', 'Parch','Fare', 'Embarked']]

#将乘客分为已知年龄和未知年龄两部分
    age_known = age_data[age_data.Age.notnull()].as_matrix()
    age_unknown = age_data[age_data.Age.isnull()].as_matrix()

#y为目标年龄，X为特征值
    y = age_known[:,0]
    X = age_known[:, 1:]

#fit到RandomForestRegressor中
    rfr = RandomForestRegressor(random_state = 0, n_estimators = 2000, n_jobs = 1)
    rfr.fit(X,y)

#用得到的模型给出年龄预测值
    pred_age = rfr.predict(age_unknown[:,1::])
#用预测数据填充缺失值
    data_train.loc[(data_train.Age.isnull()),'Age'] = pred_age
    
    return data_train, rfr


In [45]:
data_train, rfr = set_missing_ages(data_train)

In [46]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [47]:
import sklearn.preprocessing as preprocessing
scaler = preprocessing.StandardScaler()
age_scale_param = scaler.fit(data_train['Age'].reshape(-1,1))
data_train['Age_scaled'] = scaler.fit_transform(data_train['Age'].reshape(-1,1), age_scale_param)
fare_scale_param = scaler.fit(data_train['Fare'].reshape(-1,1))
data_train['Fare_scaled'] = scaler.fit_transform(data_train['Fare'].reshape(-1,1), fare_scale_param)

C:\Users\Seele\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Seele\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.
C:\Users\Seele\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """
C:\Users\Seele\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [48]:
from sklearn import linear_model

In [49]:
data_train2 = data_train.filter(regex = 'Survived|Age_.*|SibSp|Parch|Fare_.*|Embarked|Sex|Pclass')
data_train3 = data_train2.as_matrix()


In [50]:
y = list(data_train3[:, 0])
X = data_train3[:,1:]

In [51]:
clf = linear_model.LogisticRegression(C=1.0,penalty='l1', tol = 1e-6)
clf.fit(X,y)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=1e-06,
          verbose=0, warm_start=False)

In [52]:
test = pd.read_csv('C:/Users/Seele/Documents/Jupyter/KAGGLE/Titanic/test.csv')
origin_test=pd.read_csv('C:/Users/Seele/Documents/Jupyter/KAGGLE/Titanic/test.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [53]:
test['Embarked'] = test['Embarked'].fillna('S')

In [54]:
test = test.drop(['Cabin'],1)

In [55]:

test.loc[test['Embarked'] == 'S', 'Embarked'] = 0
test.loc[test['Embarked'] == 'C', 'Embarked'] = 1
test.loc[test['Embarked'] == 'Q', 'Embarked'] = 2
test.loc[test['Sex'] == 'male','Sex'] = 0
test.loc[test['Sex'] == 'female','Sex'] =1       
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

In [56]:
test,rfr = set_missing_ages(test)

In [57]:
#将测试数据归一化
test['Age_scaled'] = scaler.fit_transform(test['Age'].reshape(-1,1), age_scale_param)
fare_scale_param = scaler.fit(test['Fare'].reshape(-1,1))
test['Fare_scaled'] = scaler.fit_transform(test['Fare'].reshape(-1,1), fare_scale_param)

C:\Users\Seele\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.
C:\Users\Seele\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
C:\Users\Seele\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [58]:
test2 = test.filter(regex='Age_.*|SibSp|Parch|Fare_.*|Embarked|Sex|Pclass')
predictions = clf.predict(test2)
result = pd.DataFrame({'PassengerId':origin_test['PassengerId'].as_matrix(), 'Survived':predictions.astype(np.int32)})
result.to_csv("D:/kaggle/prediction.csv",index = False)